<a href="https://colab.research.google.com/github/zganjei/Chatbot/blob/main/chatbot_llama_fastapi_dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create chatbot for customer service using RAG (Retrieval-Augmented Generation)

## Installing required libraries

In [2]:
%%capture
!pip install openai
!pip install llama-index
!pip install --user pypdf
!pip install --user sentence_transformers
!pip install fastapi uvicorn
!pip install dash

## Setup OpenAI API key



In [3]:
import openai
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from google.colab import userdata
api_key = userdata.get('openai.api_key')
if api_key:
  print("API key: "+api_key[:10])
else:
  print("API key not found!")

openai.api_key = api_key

API key: sk-proj-5p


# Create Index  for the document having FAQ info about the company

Using LlamaIndex

Mount Google Drive that includes the file of FAQs

In [4]:
import os
from google.colab import drive

drive.mount('/content/drive')
path = "/content/drive/MyDrive/colab"

Mounted at /content/drive


Index the file and save the index for later use in folder ml_index


In [5]:
documents = SimpleDirectoryReader(path).load_data()
index = VectorStoreIndex.from_documents(documents)
storage_context = index.storage_context.persist("ml_index")

In [6]:
# storage_context = StorageContext.from_defaults(persist_dir = "ml_index")
# index = load_index_from_storage(storage_context)
#query engine
engine = index.as_query_engine()
#ask a basic question
result = engine.query("what is the file about")
print(result)

The file is about a user guide for installing acrylic nails at home, including the materials needed and a step-by-step process for preparing the nails and applying the nail tips.


# Create a REST API using FastAPI

In [7]:
%%capture
!pip install fastapi uvicorn

In [8]:
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio

class Item(BaseModel):
  question:str

app = FastAPI()
@app.post("/")
def query(item: Item):
  result = engine.query(item.question)
  return(result)

nest_asyncio.apply()

Use Uvicorn to run the API as a server

In [9]:
import uvicorn
import threading

def run():
  uvicorn.run(app,host = "0.0.0.0", port = 8000)

threading.Thread(target=run).start()

# Test the API

Client library

In [10]:
import requests

def send_question_to_api(q):
    url = "http://0.0.0.0:8000"
    data = {"question": q}

    try:
        response = requests.post(url, json=data)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

        result = response.json()["response"]
        return result

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

# Example usage:
question = "tell me about the bussiness in 20 words"
api_response = send_question_to_api(question)

if api_response:
    print(f"The answer to {question} is: {api_response}")

Error: HTTPConnectionPool(host='0.0.0.0', port=8000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7b69ebe305d0>: Failed to establish a new connection: [Errno 111] Connection refused'))


# Create user interface with Dash

In [11]:
import random
import dash
from dash import Dash, dcc, html, callback, State, Output, Input

app = Dash(__name__)
url = "http://0.0.0.0:8000"
app.layout = html.Div(children = [
    html.H1(children="Custer Service Chatbot for NailPro Supply"),
    html.Div(id="output-conversation",
             style= {"width":"90%",
                     "height":"80vh",
                     "margin" : "auto",
                     "overflow-y":"auto"}),
    html.Div(children = [
        dcc.Textarea(id = "input-text", placeholder = "Type your question here ...",
                     style = {"width":"100%"}),
        html.Button("Submit", id = "input-submit", n_clicks = 0)
        ],
       style = {"width":"90%", "margin" : "auto"}),
    dcc.Store(id="store-chat",data = "")
    ])

@callback(
    [Output("store-chat","data"), Output("input-text","value")],
    [Input("input-submit","n_clicks")],
    [State("input-text","value"), State("store-chat","data")]
)

def query_chatbot(n_clicks,input_value,chat):
  if n_clicks == 0:  # Ensure initial state works
        return "", ""
  if input_value =="" or input_value is None:
    return chat, ""
  chat = chat or ""
  chat += f"You: {input_value}<split>Bot: "
  query = chat.replace("<split>","\n").replace("Bot:", "").replace("You:","")
  result = requests.post(url, json = {"question":query})

  if result.status_code ==200:
    response = result.json()["response"]
  else:
    response = f"Error {result.reason}"
  chat += f"{response}<split>"
  return chat, ""

@app.callback(
    Output("output-conversation","children"),
    Input("store-chat","data")
)
def update_conversation(conversation):
  conversation = conversation or ""
  return [
      html.Div(message,
               style = {"max-width":"60%",
                        "width":"max-content",
                        "padding":"10px"})
      for message in conversation.split("<split>")
  ]

if __name__ == "__main__":
  app.run_server(mode = "jupyterlab",port = 8050, debug=True)


INFO:     Started server process [389]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


<IPython.core.display.Javascript object>